# Langgraph
---

## Core Design

At its core, LangGraph models agent workflows as state machines. You define the behavior of your agents using three key components:

- State: A shared data structure that represents the current snapshot of your application. It can be any Python type, but is typically a TypedDict or Pydantic BaseModel.

- Nodes: Python functions that encode the logic of your agents. They receive the current State as input, perform some computation or side-effect, and return an updated State.

- Edges: Control flow rules that determine which Node to execute next based on the current State. They can be conditional branches or fixed transitions.

By composing Nodes and Edges, you can create complex, looping workflows that evolve the State over time. The real power, though, comes from how LangGraph manages that State.

##### Or in short: nodes do the work. edges tell what to do next.

## Graph Definitions

Graphs are the core abstraction of LangGraph. Each StateGraph implementation is used to create graph workflows. Once compiled, you can run the CompiledGraph to run the application.

### StateGraph 

A graph whose nodes communicate by reading and writing to a shared state.Each state key can optionally be annotated with a reducer function that will be used to aggregate the values of that key received from multiple nodes. 

##### Parameters:

- state_schema – The schema class that defines the state.
- config_schema – The schema class that defines the configuration. Use this to expose configurable parameters in your API.

In [1]:
from langchain_core.runnables import RunnableConfig
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint import MemorySaver
from langgraph.graph import StateGraph


In [2]:
def reducer(a: list, b: int | None) -> int:
    if b is not None:
        return a + [b]
    return a


In [3]:
class State(TypedDict):
    x: Annotated[list, reducer]
        
class ConfigSchema(TypedDict):
    r: float

In [4]:
graph = StateGraph(State, config_schema=ConfigSchema)

In [5]:
def node(state: State, config: RunnableConfig) -> dict:
    r = config["configurable"].get("r", 1.0)
    x = state["x"][-1]
    next_value = x * r * (1 - x)
    return {"x": next_value}


In [6]:
graph.add_node("A", node)
graph.set_entry_point("A")
graph.set_finish_point("A")
compiled = graph.compile()

In [7]:
print(compiled.config_specs)

[ConfigurableFieldSpec(id='r', annotation=<class 'float'>, name=None, description=None, default=None, is_shared=False, dependencies=None)]


In [11]:
step1 = compiled.invoke({"x": 0.5}, {"configurable": {"r": 4.0}})

In [12]:
step1

{'x': [0.5, 1.0]}

add_conditional_edges(source, path, path_map=None, then=None) 
- Add a conditional edge from the starting node to any number of destination nodes. 

set_entry_point(key) 
- Specifies the first node to be called in the graph.

set_conditional_entry_point(path, path_map=None, then=None) 
- Sets a conditional entry point in the graph.

set_finish_point(key) 
- Marks a node as a finish point of the graph. If the graph reaches this node, it will cease execution.

add_edge(start_key, end_key) 
- Adds a directed edge from the start node to the end node.
- If the graph transitions to the start_key node, it will always transition to the end_key node next.

compile(checkpointer=None, interrupt_before=None, interrupt_after=None, debug=False)
- Compiles the state graph into a CompiledGraph object.
- The compiled graph implements the Runnable interface and can be invoked, streamed, batched, and run asynchronously.

### MessageGraph

- A StateGraph where every node receives a list of messages as input and returns one or more messages as output.
- MessageGraph is a subclass of StateGraph whose entire state is a single, append-only* list of messages.
- Each node in a MessageGraph takes a list of messages as input and returns zero or more messages as output. 
- The add_messages function is used to merge the output messages from each node into the existing list of messages in the graph's state.

In [15]:
from langgraph.graph.message import MessageGraph
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

In [16]:
# Example 1: Simple Chatbot Response
builder = MessageGraph()
builder.add_node("chatbot", lambda state: [("assistant", "Hello!")])
builder.set_entry_point("chatbot")
builder.set_finish_point("chatbot")
response = builder.compile().invoke([("user", "Hi there.")])
print(response)

[HumanMessage(content='Hi there.', id='45fea036-a188-4a41-b5eb-387c391715b0'), AIMessage(content='Hello!', id='d5b0ab43-d0fd-431b-a9d9-3c1ef53df0df')]


In [17]:
# Example 2: Chatbot with Tool Call
builder = MessageGraph()
builder.add_node(
    "chatbot",
    lambda state: [
        AIMessage(
            content="Hello!",
            tool_calls=[{"name": "search", "id": "123", "args": {"query": "X"}}],
        )
    ],
)
builder.add_node(
    "search",
    lambda state: [ToolMessage(content="Searching...", tool_call_id="123")]
)
builder.set_entry_point("chatbot")
builder.add_edge("chatbot", "search")
builder.set_finish_point("search")
response = builder.compile().invoke([HumanMessage(content="Hi there. Can you search for X?")])
print(response)

[HumanMessage(content='Hi there. Can you search for X?', id='5592b299-1a47-498e-b251-e6b67701b920'), AIMessage(content='Hello!', id='b32db576-1fcb-4bc1-8360-dbf04b9e5619', tool_calls=[{'name': 'search', 'args': {'query': 'X'}, 'id': '123'}]), ToolMessage(content='Searching...', id='cb63df34-d5bf-4a05-8d71-3cd66261a9c9', tool_call_id='123')]


### Constants

The following constants and classes are used to help control graph execution.

#### START

- START is a string constant ("__start__") that serves as a "virtual" node in the graph. Adding an edge (or conditional edges) from START to node one or more nodes in your graph will direct the graph to begin execution there.


In [19]:
from langgraph.graph import START
# builder.add_edge(START, "my_node")
# Or to add a conditional starting point
# builder.add_conditional_edges(START, my_condition)

#### END
- END is a string constant ("__end__") that serves as a "virtual" node in the graph. Adding an edge (or conditional edges) from one or more nodes in your graph to the END "node" will direct the graph to cease execution as soon as it reaches this point.

In [20]:
from langgraph.graph import END

builder.add_edge("my_node", END) # Stop any time my_node completes
# Or to conditionally terminate
def my_condition(state):
    if state["should_stop"]:
        return END
    return "my_node"
builder.add_conditional_edges("my_node", my_condition)


Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


### Send
- A message or packet to send to a specific node in the graph.

- The Send class is used within a StateGraph's conditional edges to dynamically route states to different nodes based on certain conditions. This enables creating "map-reduce" like workflows, where a node can be invoked multiple times in parallel on different states, and the results can be aggregated back into the main graph's state.

In [21]:
from typing import Annotated, TypedDict
import operator
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph

In [22]:
# Define the OverallState TypedDict with annotated jokes list
class OverallState(TypedDict):
    subjects: list[str]
    jokes: Annotated[list[str], operator.add]

# Define the function to continue to jokes based on the state
def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": s}) for s in state['subjects']]

In [23]:
# Initialize the StateGraph with OverallState
builder = StateGraph(OverallState)

# Add a node that generates a joke based on the subject in the state
builder.add_node("generate_joke", lambda state: {"jokes": [f"Joke about {state['subject']}"]})

# Add conditional edges from START to continue_to_jokes function
builder.add_conditional_edges(START, continue_to_jokes)

# Add an edge from "generate_joke" to END
builder.add_edge("generate_joke", END)

In [24]:
# Compile the graph
graph = builder.compile()

# Invoke the graph with initial state
result = graph.invoke({"subjects": ["cats", "dogs"]})

In [25]:
print(result)

{'subjects': ['cats', 'dogs'], 'jokes': ['Joke about cats', 'Joke about dogs']}
